In [1]:
########## author: vasigiurgi #############


import os
import sys
from ev_utils import read_path, zero_unpadding, clu_to_local, mainEval
from utils_evaluation import split_check, kittiroad_camera
from pathlib import Path
base_path = Path.cwd().parent 
# Libs path
sys.path.insert(0, str(base_path / 'model_arch' / 'libs'))
import numpy as np
import cv2
from tensorflow import keras
from ds_layer_p2p_RT import DS1, DS1_activate, DS2, DS2_omega, DS3_Dempster, DS3_normalize, SelectSingleton, BeliefIntervalDistance, BeliefPlausibility, BeliefPlausibilityFocused # Ensure your custom layer module is imported
import tensorflow as tf


In [2]:

# ===========================================
# Required only on the local gpu rtx 2070 (may be due to some memory issue)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Memory settings
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----
#============================================


2024-07-05 12:08:24.733590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:24.737976: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:24.738092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:24.886225: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9



2024-07-05 12:08:25.176624: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:25.176757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:25.176856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-05 12:08:25.176955: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
# cluster directory
input_dir_rgb = str(base_path / 'image_2')
target_dir = str(base_path / 'gt_road')
model_dir= str(base_path / 'model_arch' / 'model_camera_rd_evi')
file_dir= str(base_path / 'output')
out_dir= str(base_path / 'evaluation' / 'prediction_mask_evi')

def main():
    
    # List of validation split
    val_input_img_paths_rgb= read_path(file_dir,'val_input_img_paths_rgb.txt')
    val_target_img_paths=read_path(file_dir, 'val_target_img_paths.txt')
    
    # From cluster to local
    for i in range(len(val_input_img_paths_rgb)):
        val_input_img_paths_rgb[i]=clu_to_local(val_input_img_paths_rgb[i])
        val_target_img_paths[i]= clu_to_local(val_target_img_paths[i])
    
    # Prepare split matching
    val_cam= split_check(val_input_img_paths_rgb)
    val_target= split_check(val_target_img_paths)
        
    # Instantiate data Sequences 
    batch_size=1
    img_size = (384,1248)
    val_input_gen= kittiroad_camera(batch_size, img_size, val_input_img_paths_rgb, val=True)
    
    # Load the model architecture from the JSON file
    with open(os.path.join(model_dir, 'model_camera_rd_evi.json'), 'r') as json_file:
        model_json = json_file.read()
    custom_objects = {
    'DS1': DS1,
    'DS1_activate': DS1_activate,
    'DS2': DS2,
    'DS2_omega': DS2_omega,
    'DS3_Dempster': DS3_Dempster,
    'DS3_normalize':DS3_normalize, 
    'SelectSingleton':SelectSingleton,
    'BeliefIntervalDistance':BeliefIntervalDistance,
    'BeliefPlausibility':BeliefPlausibility,
    'BeliefPlausibilityFocused':BeliefPlausibilityFocused}
    model = keras.models.model_from_json(model_json, custom_objects)

    # Load the saved weights into the model
    model.load_weights(os.path.join(file_dir, 'model_camera_rd_prob_weights'))
    # Prediction
    
    # Write prediction to file
    val_samples=len(val_input_img_paths_rgb)

    for i in range(val_samples):
        val_preds = model.predict(val_input_gen[i])
        img = cv2.imread(val_input_img_paths_rgb[i])
        h=img.shape[0]
        w=img.shape[1]
        unpadded_argmax_map=zero_unpadding(val_preds[0], h, w, img_size)
        #pdb.set_trace()
        argmax_map = np.array(unpadded_argmax_map)
        argmax_map=np.argmin(argmax_map,axis=-1)
        #argmax_map=np.argmax(argmax_map,axis=-1)
        fname= os.path.split(val_target_img_paths[i])[1]
        pred_path= os.path.join(out_dir, fname )
        cv2.imwrite(pred_path, argmax_map*127)   
        
if __name__=="__main__":
    main()

2024-07-05 12:09:02.236213: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2024-07-05 12:09:02.657642: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-05 12:09:02.658349: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-05 12:09:02.658364: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-07-05 12:09:02.658896: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-05 12:09:02.658941: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
